# Code to Scrape DARKO and BPM
### Credits to Krishna Narsu for the DARKO R code

In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

export_DIR1 = "../fdata/bbref/"
export_DIR2 = "../fdata/all_in_one_metrics/"

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
df_R = []

In [ ]:
bpm,dpm,dpmhistory = [],[],[]

In [ ]:
%%R -o bpm,dpm,dpmhistory
library(tidyverse)
library(nbastatR)
library(googlesheets4)
library(googledrive)
library(tidyverse)

Sys.setenv("VROOM_CONNECTION_SIZE" = 131072 * 2)
season <- 2024

bpm <- bref_players_stats(
  seasons = season,
  tables = "advanced",
  include_all_nba = FALSE,
  only_totals = FALSE,
  nest_data = FALSE,
  assign_to_environment = TRUE,
  widen_data = TRUE,
  join_data = TRUE,
  return_message = TRUE
)

#Drive authentication
drive_auth(email = "sra.djoker@gmail.com")
#DPM
dpm <- googlesheets4::read_sheet(ss = "1mhwOLqPu2F9026EQiVxFPIN1t9RGafGpl-dokaIsm9c",
                                 sheet = "Current DPM Ranks") %>% data.frame()

dpmhistory <- googlesheets4::read_sheet(ss = "1mhwOLqPu2F9026EQiVxFPIN1t9RGafGpl-dokaIsm9c",
                                        sheet = "Full DPM History") %>% data.frame()

In [ ]:
bpm.to_csv(export_DIR1 + "NBA_bbref_Player_Adv_2023.csv")
bpm.to_parquet(export_DIR1 + "NBA_bbref_Player_Adv_2023.parquet")
dfb = bpm[["idPlayerNBA",'ratioOBPM', 'ratioDBPM', 'ratioBPM']].reset_index(drop=True)
dfb["idPlayerNBA"] = dfb["idPlayerNBA"].astype(int)
dfb = dfb.query("idPlayerNBA > 0")

In [ ]:
dfd= dpm.rename(columns={"NBA.ID":"idPlayerNBA","Player.Name":"namePlayer"})
dfd.columns = ['idPlayerNBA', 'namePlayer', 'Position', 'age', 'dpm','o_dpm', 'd_dpm', 'box_odpm', 'box_ddpm', 'on_off_odpm', 'on_off_ddpm']
dfd["idPlayerNBA"] = dfd["idPlayerNBA"].astype(int)
dfd.columns = dfd.columns.str.replace("Offensive","O")
dfd.columns = dfd.columns.str.replace("Defensive","D")
dfd.columns = dfd.columns.str.replace(".","-")
dfd.to_csv(export_DIR2 + "NBA_DARKO_Current.csv")
dfd.to_parquet(export_DIR2 + "NBA_DARKO_Current.parquet")
dfd = dfd.drop(columns=["Position"])

In [ ]:
df1 = pd.merge(dfd,dfb, on = "idPlayerNBA", how="right")

In [ ]:
print(len(dfd))
print(len(dfb))
print(len(df1))

In [ ]:
dfdh= dpmhistory.rename(columns={"nba_id":"idPlayerNBA","player_name":"namePlayer"})
dfdh["idPlayerNBA"] = dfdh["idPlayerNBA"].astype(int)
dfdh.to_csv(export_DIR2 + "NBA_DARKO_History.csv")
dfdh.to_parquet(export_DIR2 + "NBA_DARKO_History.parquet")
dfdh =dfdh.drop(columns=["tm_id","team_name"])
dfdh = dfdh.query("season == 2024")
dfdh = dfdh.drop(columns=["season"])

In [ ]:
nan_rows = df1.isna().any(axis=1)

In [ ]:
missing_players = df1["idPlayerNBA"][nan_rows].to_list()

In [ ]:
df2 = dfdh[dfdh["idPlayerNBA"].isin(missing_players)].reset_index(drop=True)

In [ ]:
df1nan = df1[nan_rows].sort_values("idPlayerNBA").reset_index(drop=True)
df1nan = df1nan.drop(columns = [ 'namePlayer', 'age', 'dpm','o_dpm', 'd_dpm', 'box_odpm', 'box_ddpm', 'on_off_odpm', 'on_off_ddpm'])

In [ ]:
df1n = df1.dropna()

In [ ]:
df3 = pd.merge(df2,df1nan,on="idPlayerNBA")

In [ ]:
df4 = pd.concat([df1n,df3])

In [ ]:
df4.to_csv(export_DIR2 + "NBA_DARKO_Processed.csv")